In [1]:
#!/usr/bin/env python

import sys
import string, re
import pandas as pd

In [2]:
class Zhuyin2IPA:
    def __init__(self):
        self.SPECZ1=['ㄓ','ㄔ','ㄕ','ㄖ','ㄗ','ㄘ','ㄙ']
        self.SPECZ2=['ㄧ','ㄨ','ㄩ']
        self.CSPECZ2=['i','u','y']
        self.FOURSOUNDS=['ˊ','ˇ','ˋ','˙']
        self.DUOZ=['ㄧㄢ','ㄩㄢ','ㄧㄣ','ㄧㄥ','ㄨㄥ','ㄩㄥ', 'ㄧㄚ','ㄧㄢ','ㄨㄟ','ㄨㄚ','ㄩㄣ','ㄨㄛ','ㄨㄢ','ㄨㄣ','ㄨㄞ',
                   'ㄧㄠ','ㄧㄡ','ㄧㄤ','ㄧㄝ','ㄩㄝ','ㄨㄤ']
        
        self.zdf= pd.read_excel("zhuyin2ipa2syll_comma.xlsx", engine='openpyxl')

    def is_ascii(self, check_str):
         return all(ord(c) < 128 for c in check_str)

    def foursounds(self, c):
        if 'ˋ' in c:
            return "51"
        elif 'ˊ' in c:
            return "35"
        elif 'ˇ' in c:
            return "21"
        elif '˙' in c:
            return "0"
        else:
            return "55"
    
    def convertZH(self, zh):
        #print("searching", zh)
        x = self.zdf[self.zdf['Zhuyin']==zh].IPA.to_string(index=False)
        if x == "Series([], )":
            return ''
        else:
            return x.replace(' ','')

    def remove_foursounds(self, zh):
        z = zh
        for f in self.FOURSOUNDS:
            z = z.replace(f,'')
        return z

    def is_duo(self, zh):
        for d in self.DUOZ:
            if d in zh:
                #print("Is duo")
                return True
        #print("Is not duo")
        return False

    def duo_convert(self, zh):
        #print("zh0",zh[0])
        matching = False
        for z in self.SPECZ2:
            #print("zzh", z, zh[0])
            if z == zh[0]:
                matching = True
                break

        #if matching:
            #print("Converting zh", zh)
        #    return self.convertZH(zh)
        #else:
        #    return self.convertZH(zh[0]) + self.convertZH(zh[1:]).replace('\n','')
        if matching:
            #print("Converting zh", zh)
            return [self.convertZH(zh)]
        else:
            #print("Converting zh", zh[0], zh[1:])
            return [self.convertZH(zh[0]), self.convertZH(zh[1:]).replace('\n','')]

    def convertIUY(self, z):
        if z == 'ㄧ':
            return 'i'
        elif z == 'ㄨ':
            return 'u'
        else:
            return 'y'

    def zhuyin2ipa(self, zhuyin):
        zh=self.remove_foursounds(zhuyin)
        idx = 0
        zhlen = len(zh)

        if self.is_duo(zh):
            return(self.duo_convert(zh))

        IPAL=[]
        while (idx < zhlen):
            #print(zh[idx])
            if zh[idx] in self.SPECZ1:
                if zhlen == 1:
                    return ([self.convertZH(zh[idx])+',ɨ'])
                else:
                    IPAL.append(self.convertZH(zh[idx]))
                    idx = idx + 1

            elif zh[idx] in self.SPECZ2:
                if idx == zhlen - 1:
                    #The last one
                    lc = self.convertIUY(zh[idx])
                    IPAL.append(lc)
                    return IPAL
                else:
                    IPAL.append(self.convertZH(zh[idx]))
                    idx = idx + 1
            else:
                IPAL.append(self.convertZH(zh[idx]))
                idx = idx + 1

        #print(IPAL)
        return IPAL
    
    def ipa2cgvn(self, ipa):
        ipalen = len(ipa)
        idx = 0
        cgvx = []
        
        print(ipa[idx])        
        while (idx < ipalen):

            x=self.zdf[self.zdf['IPA'] == ipa[idx]]
            print(x)
            xx = x['Syllable'].to_string(index=False).replace(" ","").split(",")
            cgvx.append(xx)
            idx = idx + 1
        
        xcgvx=[]
        for xx in cgvx:
            for xi in xx:
                xcgvx.append(xi)
            
        return xcgvx

    def ipaa2cgvn(self, ipaa):
        x=self.zdf[self.zdf['IPA'] == ipaa]
        #print(x)
        xx = x['Syllable'].to_string(index=False).replace(" ","").split(",")

        return xx        
        
z = Zhuyin2IPA()
z.zhuyin2ipa('ㄕˊ')

#.ipa2cgvn(['ʂ','ɨ'])

['ʂ,ɨ']

In [19]:
TITLE=["Revised_target","Learner output"]

class thai_dataset:
    def __init__(self):
        self.tdf= pd.read_excel("thai-student_edit distance_revised 07312023.xlsx", engine='openpyxl')
        self.zy=Zhuyin2IPA()
        
    def sheet1_process(self):
        print(TITLE)
        for i in range(self.tdf.shape[0]):
            for j in TITLE:
                chn=self.tdf.loc[i,j]
                chnl=chn.replace(" ","").replace("[","").replace("]","").replace("'","").split(",")
                print(chnl, end=" ")
                
                for item in chnl:
                    print(self.zy.ipaa2cgvn(item), end="")
                print("|", end="")
            print()
    
    def sheet3_process(self):
        return
            
x = thai_dataset()
x.sheet1_process()

['Revised_target', 'Learner output']
['i'] ['V']|['i'] ['V']|
['i'] ['V']|['i'] ['V']|
['i'] ['V']|['i'] ['V']|
['i'] ['V']|['i'] ['V']|
['i'] ['V']|['i'] ['V']|
['i'] ['V']|['i'] ['V']|
['i'] ['V']|['i'] ['V']|
['i'] ['V']|['i'] ['V']|
['i'] ['V']|['i'] ['V']|
['i'] ['V']|['i'] ['V']|
['i'] ['V']|['i'] ['V']|
['i', 'tʰ', 'j', 'ɛ', 'n'] ['V']['C']['G']['V']['C']|['i', 'tʰ', 'j', 'ɛ', 'n'] ['V']['C']['G']['V']['C']|
['i', 'tʰ', 'j', 'ɛ', 'n'] ['V']['C']['G']['V']['C']|['i', 'tʰ', 'j', 'ɛ', 'n'] ['V']['C']['G']['V']['C']|
['i', 'tʰ', 'j', 'ɛ', 'n'] ['V']['C']['G']['V']['C']|['i', 'tʰ', 'j', 'ɛ', 'n'] ['V']['C']['G']['V']['C']|
['i', 'tʰ', 'j', 'ɛ', 'n'] ['V']['C']['G']['V']['C']|['i', 'tʰ', 'j', 'ɛ', 'n'] ['V']['C']['G']['V']['C']|
['i', 'tʰ', 'j', 'ɛ', 'n'] ['V']['C']['G']['V']['C']|['i', 'tʰ', 'i', 'ɛ', 'n'] ['V']['C']['V']['V']['C']|
['i', 'tʰ', 'j', 'ɛ', 'n'] ['V']['C']['G']['V']['C']|['i', 'tʰ', 'j', 'ɛ', 'n'] ['V']['C']['G']['V']['C']|
['i', 'tʰ', 'j', 'ɛ', 'n'] ['V']['C']['G']['V'

['ʂ', 'ɨ'] ['C']['V']|['ʂ', 'ɨ'] ['C']['V']|
['ʂ', 'ɨ'] ['C']['V']|['ʂ', 'ɨ'] ['C']['V']|
['ʂ', 'ɨ'] ['C']['V']|['ʂ', 'ɨ'] ['C']['V']|
['ʂ', 'ɨ', 'i'] ['C']['V']['V']|['ʂ', 'ɨ', 'i'] ['C']['V']['V']|
['ʂ', 'ɨ', 'i'] ['C']['V']['V']|['ʂ', 'ɨ', 'i'] ['C']['V']['V']|
['ʂ', 'ɨ', 'ɚ'] ['C']['V']['V']|['tʂ', 'ɨ', 'ɚ'] ['C']['V']['V']|
['ʂ', 'ɨ', 'ɚ'] ['C']['V']['V']|['ʂ', 'ɨ', 'ɚ'] ['C']['V']['V']|
['ʂ', 'ɨ', 'ɚ'] ['C']['V']['V']|['ʂ', 'ɨ', 'ɚ'] ['C']['V']['V']|
['ʂ', 'ɨ', 'ɚ'] ['C']['V']['V']|['ʂ', 'ɨ', 'ɚ'] ['C']['V']['V']|
['ʂ', 'ɨ', 's', 'a', 'n'] ['C']['V']['C']['V']['C']|['ʂ', 'ɨ', 'ɕ', 'a', 'n'] ['C']['V']['C']['V']['C']|
['ʂ', 'ɨ', 's', 'a', 'n'] ['C']['V']['C']['V']['C']|['ʂ', 'ɨ', 's', 'a', 'n'] ['C']['V']['C']['V']['C']|
['ʂ', 'ɨ', 's', 'a', 'n', 'k', 'ɤ', 'ʐ', 'ə', 'n'] ['C']['V']['C']['V']['C']['C']['V']['C']['V']['C']|['tʂʰ', 'ɨ', 's', 'a', 'n', 'k', 'ɤ', 'ʐ', 'ə', 'n'] ['C']['V']['C']['V']['C']['C']['V']['C']['V']['C']|
['ʂ', 'ɨ', 't', 'j', 'ɛ', 'n'] ['C']['V']['C']['G']['V'][

['ɕ', 'j', 'a'] ['C']['G']['V']|['tɕ', 'i', 'a'] ['C']['V']['V']|
['ɕ', 'j', 'a'] ['C']['G']['V']|['x', 'i', 'a'] ['C']['V']['V']|
['ɕ', 'j', 'a'] ['C']['G']['V']|['ɕ', 'j', 'a'] ['C']['G']['V']|
['ɕ', 'j', 'a'] ['C']['G']['V']|['ɕ', 'j', 'a'] ['C']['G']['V']|
['ɕ', 'j', 'a'] ['C']['G']['V']|['ɕ', 'j', 'a'] ['C']['G']['V']|
['ɕ', 'j', 'a'] ['C']['G']['V']|['ɕ', 'j', 'a'] ['C']['G']['V']|
['ɕ', 'j', 'a'] ['C']['G']['V']|['ɕ', 'j', 'a'] ['C']['G']['V']|
['ɕ', 'j', 'a'] ['C']['G']['V']|['tɕ', 'i', 'a'] ['C']['V']['V']|
['ɕ', 'j', 'a'] ['C']['G']['V']|['x', 'i', 'a'] ['C']['V']['V']|
['ɕ', 'j', 'a'] ['C']['G']['V']|['ɕ', 'j', 'a'] ['C']['G']['V']|
['ɕ', 'j', 'a'] ['C']['G']['V']|['ɕ', 'j', 'a'] ['C']['G']['V']|
['ɕ', 'j', 'a'] ['C']['G']['V']|['ɕ', 'j', 'a'] ['C']['G']['V']|
['ɕ', 'j', 'a'] ['C']['G']['V']|['ɕ', 'j', 'a'] ['C']['G']['V']|
['ɕ', 'j', 'a'] ['C']['G']['V']|['ɕ', 'j', 'a'] ['C']['G']['V']|
['ɕ', 'j', 'a'] ['C']['G']['V']|['tɕ', 'i', 'a'] ['C']['V']['V']|
['ɕ', 'j', 'a'] ['C'][

['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|['tɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|
['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|['tɕʰ', 'j', 'a', 'u'] ['C']['G']['V']['V']|
['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|
['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|
['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|['tɕʰ', 'i', 'a', 'u'] ['C']['V']['V']['V']|
['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|['ɕ', 'i', 'a', 'u'] ['C']['V']['V']['V']|
['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|['tɕʰ', 'j', 'a', 'u'] ['C']['G']['V']['V']|
['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|['tɕʰ', 'i', 'a', 'u'] ['C']['V']['V']['V']|
['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|['tɕʰ', 'j', 'a', 'u'] ['C']['G']['V']['V']|
['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|['tɕʰ', 'i', 'a', 'u'] ['C']['V']['V']['V']|
['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|['tɕʰ', 'j', 'a', 'y'] ['C']['G']['V']['V']|
['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|['ɕ', 'i

['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|['tɕʰ', 'i', 'a', 'u'] ['C']['V']['V']['V']|
['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|['tɕʰ', 'i', 'a', 'u'] ['C']['V']['V']['V']|
['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|['tɕʰ', 'i', 'a', 'u'] ['C']['V']['V']['V']|
['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|
['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|
['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|['tɕʰ', 'i', 'a', 'u'] ['C']['V']['V']['V']|
['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|['tɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|
['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|['tɕʰ', 'j', 'a', 'u'] ['C']['G']['V']['V']|
['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|
['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|
['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|['tɕʰ', 'i', 'a', 'u'] ['C']['V']['V']['V']|
['ɕ', 'j', 'a', 'u'] ['C']['G']['V']['V']|['ɕ', 'i',

...

['tʰ', 'a', 'ŋ', 'k', 'w', 'ɔ'] ['C']['V']['N']['C']['G']['V']|['tʰ', 'a', 'ŋ', 'k', 'w', 'ɔ'] ['C']['V']['N']['C']['G']['V']|
['tʰ', 'a', 'ŋ', 'k', 'w', 'ɔ'] ['C']['V']['N']['C']['G']['V']|['tʰ', 'a', 'ŋ', 'k', 'w', 'a'] ['C']['V']['N']['C']['G']['V']|
['tʰ', 'a', 'ŋ', 'k', 'w', 'ɔ'] ['C']['V']['N']['C']['G']['V']|['tʰ', 'a', 'n', 'k', 'w', 'ɔ'] ['C']['V']['C']['C']['G']['V']|
['tʰ', 'a', 'ŋ', 'k', 'w', 'ɔ'] ['C']['V']['N']['C']['G']['V']|['tʰ', 'a', 'n', 'k', 'w', 'ɔ'] ['C']['V']['C']['C']['G']['V']|
['tʰ', 'a', 'ŋ', 'k', 'w', 'ɔ'] ['C']['V']['N']['C']['G']['V']|['tʰ', 'a', 'n', 'k', 'u', 'a'] ['C']['V']['C']['C']['V']['V']|
['tʰ', 'a', 'ŋ', 'k', 'w', 'ɔ'] ['C']['V']['N']['C']['G']['V']|['tʰ', 'a', 'n', 'k', 'w', 'a'] ['C']['V']['C']['C']['G']['V']|
['tʰ', 'a', 'ŋ', 'k', 'w', 'ɔ'] ['C']['V']['N']['C']['G']['V']|['tʰ', 'a', 'n', 'k', 'w', 'a'] ['C']['V']['C']['C']['G']['V']|
['tʰ', 'a', 'ŋ', 'k', 'w', 'ɔ'] ['C']['V']['N']['C']['G']['V']|['tʰ', 'a', 'ŋ', 'k', 'w', 'a'] ['C']['V']['N'][

['t', 'j', 'ɛ', 'n'] ['C']['G']['V']['C']|['t', 'i', 'a', 'ŋ'] ['C']['V']['V']['N']|
['t', 'j', 'ɛ', 'n'] ['C']['G']['V']['C']|['t', 'i', 'a', 'ŋ'] ['C']['V']['V']['N']|
['t', 'j', 'ɛ', 'n'] ['C']['G']['V']['C']|['t', 'i', 'ɛ', 'n'] ['C']['V']['V']['C']|
['t', 'j', 'ɛ', 'n'] ['C']['G']['V']['C']|['t', 'j', 'ɛ', 'n'] ['C']['G']['V']['C']|
['t', 'j', 'ɛ', 'n'] ['C']['G']['V']['C']|['t', 'j', 'ɛ', 'n'] ['C']['G']['V']['C']|
['t', 'j', 'ɛ', 'n'] ['C']['G']['V']['C']|['t', 'i', 'a', 'ŋ'] ['C']['V']['V']['N']|
['t', 'j', 'ɛ', 'n'] ['C']['G']['V']['C']|['t', 'i', 'a', 'ŋ'] ['C']['V']['V']['N']|
['t', 'j', 'ɛ', 'n'] ['C']['G']['V']['C']|['t', 'i', 'ɛ', 'n'] ['C']['V']['V']['C']|
['t', 'j', 'ɛ', 'n'] ['C']['G']['V']['C']|['t', 'j', 'ɛ', 'n'] ['C']['G']['V']['C']|
['t', 'j', 'ɛ', 'n'] ['C']['G']['V']['C']|['t', 'j', 'ɛ', 'n'] ['C']['G']['V']['C']|
['t', 'j', 'ɛ', 'n'] ['C']['G']['V']['C']|['t', 'i', 'a', 'ŋ'] ['C']['V']['V']['N']|
['t', 'j', 'ɛ', 'n'] ['C']['G']['V']['C']|['t', 'i', 'a', 'ŋ'] ['

['w', 'a', 'n', 'ʂ', 'a', 'ŋ', 'j', 'a', 'w', 'ʂ', 'w', 'e', 'j', 'tɕ', 'j', 'a', 'w'] ['G']['V']['C']['C']['V']['N']['G']['V']['G']['C']['G']['V']['G']['C']['G']['V']['G']|['w', 'a', 'n', 'tsʰ', 'a', 'ŋ', 'j', 'a', 'w', 'ʂ', 'w', 'e', 'j', 'tɕ', 'j', 'a', 'w'] ['G']['V']['C']['C']['V']['N']['G']['V']['G']['C']['G']['V']['G']['C']['G']['V']['G']|
['w', 'a', 'n', 'ʂ', 'a', 'ŋ', 'j', 'a', 'w', 'ʂ', 'w', 'e', 'j', 'tɕ', 'j', 'a', 'w'] ['G']['V']['C']['C']['V']['N']['G']['V']['G']['C']['G']['V']['G']['C']['G']['V']['G']|['w', 'a', 'n', 'tsʰ', 'a', 'ŋ', 'j', 'a', 'w', 'tsʰ', 'w', 'e', 'j', 'ɕ', 'j', 'a', 'w'] ['G']['V']['C']['C']['V']['N']['G']['V']['G']['C']['G']['V']['G']['C']['G']['V']['G']|
['w', 'a', 'n', 'ʂ', 'a', 'ŋ', 'j', 'a', 'w', 'ʂ', 'w', 'e', 'j', 'tɕ', 'j', 'a', 'w'] ['G']['V']['C']['C']['V']['N']['G']['V']['G']['C']['G']['V']['G']['C']['G']['V']['G']|['w', 'a', 'n', 'ʂ', 'a', 'ŋ', 'i', 'a', 'w', 'ts', 'w', 'e', 'j', 'tɕ', 'i', 'a', 'w'] ['G']['V']['C']['C']['V']['N']['V']['V']

['w', 'a', 'n', 'ʂ', 'a', 'ŋ', 'j', 'a', 'w', 'ʂ', 'w', 'e', 'j', 'tɕ', 'j', 'a', 'w'] ['G']['V']['C']['C']['V']['N']['G']['V']['G']['C']['G']['V']['G']['C']['G']['V']['G']|['w', 'a', 'n', 'ʂ', 'a', 'ŋ', 'j', 'a', 'w', 'ʂ', 'w', 'e', 'j', 'tɕ', 'ɥ', 'ɛ'] ['G']['V']['C']['C']['V']['N']['G']['V']['G']['C']['G']['V']['G']['C']['G']['V']|
['w', 'a', 'n', 'ʂ', 'a', 'ŋ', 'j', 'a', 'w', 'ʂ', 'w', 'e', 'j', 'tɕ', 'j', 'a', 'w'] ['G']['V']['C']['C']['V']['N']['G']['V']['G']['C']['G']['V']['G']['C']['G']['V']['G']|['w', 'a', 'n', 'ʂ', 'a', 'n', 'j', 'a', 'w', 'ts', 'w', 'e', 'j', 'tɕ', 'ɥ', 'ɛ'] ['G']['V']['C']['C']['V']['C']['G']['V']['G']['C']['G']['V']['G']['C']['G']['V']|
['w', 'a', 'n', 'ʂ', 'a', 'ŋ', 'j', 'a', 'w', 'ʂ', 'w', 'e', 'j', 'tɕ', 'j', 'a', 'w'] ['G']['V']['C']['C']['V']['N']['G']['V']['G']['C']['G']['V']['G']['C']['G']['V']['G']|['w', 'a', 'n', 'ʂ', 'a', 'ŋ', 'j', 'a', 'w', 'ts', 'w', 'e', 'j', 'tɕ', 'j', 'a'] ['G']['V']['C']['C']['V']['N']['G']['V']['G']['C']['G']['V']['G']['C'

['w', 'a', 'n', 'ʂ', 'a', 'ŋ', 'j', 'a', 'w', 'ʂ', 'w', 'e', 'j', 'tɕ', 'j', 'a', 'w'] ['G']['V']['C']['C']['V']['N']['G']['V']['G']['C']['G']['V']['G']['C']['G']['V']['G']|['w', 'a', 'n', 'ʂ', 'a', 'ŋ', 'j', 'a', 'w', 'ʂ', 'w', 'e', 'j', 'tɕ', 'ɥ', 'ɛ'] ['G']['V']['C']['C']['V']['N']['G']['V']['G']['C']['G']['V']['G']['C']['G']['V']|
['w', 'a', 'n', 'ʂ', 'a', 'ŋ', 'j', 'a', 'w', 'ʂ', 'w', 'e', 'j', 'tɕ', 'j', 'a', 'w'] ['G']['V']['C']['C']['V']['N']['G']['V']['G']['C']['G']['V']['G']['C']['G']['V']['G']|['w', 'a', 'n', 'ʂ', 'a', 'n', 'j', 'a', 'w', 'ts', 'w', 'e', 'j', 'tɕ', 'ɥ', 'ɛ'] ['G']['V']['C']['C']['V']['C']['G']['V']['G']['C']['G']['V']['G']['C']['G']['V']|
['w', 'a', 'n', 'ʂ', 'a', 'ŋ', 'j', 'a', 'w', 'ʂ', 'w', 'e', 'j', 'tɕ', 'j', 'a', 'w'] ['G']['V']['C']['C']['V']['N']['G']['V']['G']['C']['G']['V']['G']['C']['G']['V']['G']|['w', 'a', 'n', 'ʂ', 'a', 'ŋ', 'j', 'a', 'w', 'ts', 'w', 'e', 'j', 'tɕ', 'j', 'a'] ['G']['V']['C']['C']['V']['N']['G']['V']['G']['C']['G']['V']['G']['C'

In [ ]:
class CDict:
    def __init__(self):
        self.cdict = pd.read_csv('cdict_cond.csv')
        self.z = Zhuyin2IPA()
        
    def lookup(self, c):
        zy = self.cdict[self.cdict['word'] == c]['phone'].values.tolist()
        #print(zy, end="")
        if len(zy) != 0:
            aipalx = []
            cgvnlx = []
            for zzy in zy:
                #print(zzy)
                zylist = zzy.split('　')
                #print(">>", zylist)
                for zz in zylist:
                    #print(zz)
                    ipal = self.z.zhuyin2ipa(zz)
                    ipalx = [x for x in ipal if x != '']
                    fsnd = self.z.foursounds(zz)
                    #print(",", zz, ",", ipalx, fsnd, end=" ")
                    
                    cgvx = self.z.ipa2cgvn(ipalx)
                    #print(cgvx)
                    cgvnlx.append(cgvx)
                    ipalx.append(fsnd)
                    #print()
                
                xpalx=[]
                for xx in ipalx:
                    xlst=xx.split(",")
                    for xi in xlst:
                        xpalx.append(xi)
                    
                #print(">>", xpalx)
                
                aipalx.append(xpalx)
        else:
            aipalx = []
        
        return aipalx, cgvnlx
    
c = CDict()
#c.lookup('中')
#c.lookup('國')
c.lookup('回')
#c.lookup('報')

In [ ]:
TITLE=["PRECEDING_1", "PRECEDING_2", "PRECEDING_3", "PRECEDING_4", "PRECEDING_5", 
       "FOLLOWING_1", "FOLLOWING_2", "FOLLOWING_3", "FOLLOWING_4", "FOLLOWING_5"]
ALPHA="abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
NOTATION=",.」"
num="0123456789"
cnum="零一二三四五六七八九"

class dataset:
    def __init__(self):
        self.zdf= pd.read_excel("speechdata_updated.xlsx", engine='openpyxl')
        
    def process(self):
        for i in range(self.zdf.shape[0]):
            print(i+2, end=": ")
            for j in TITLE:
                chn=self.zdf.loc[i,j]
                if not pd.isnull(chn) and not chn in ALPHA and not chn in NOTATION and chn != '':
                    if chn in num:
                        idx=num.find(chn)
                        chn = cnum[idx]
                    #print(self.zdf.loc[i,j], end="")
                    #continue
                    print(chn, end="")
                    xi, yi = c.lookup(chn)
                    print (xi, yi)
#                 elif not pd.isnull(chn):
#                     print(chn, end="")
            print()
            
x = dataset()
x.process()